# Metrics II Part 2 PS 3
Alternatively to the formulation from the lecture
$$A(\theta)X_t = B(\theta)\mathbb{E}\left[X_{t+1} \right] + C(\theta)X_{t-1} + D(\theta)\eta_t$$ with
$$X_t = \left[x_t \, \pi_t \, i_t \, g_t \, u_t\right]'$$ $$\eta_t = \left[\epsilon_{it} \, \epsilon_{gt} \, \epsilon_{ut} \right]'$$
I follow Chris Sims approach (see http://sims.princeton.edu/yftp/gensys/LINRE3A.pdf), which is coded up in FRBNY's DSGE package.
## (a)

In [1]:
using DSGE

σ = 1; κ = 0.2; β = 0.99; ϕ_π = 1.25; ϕ_x = 0.25; ρ_i = 0.5; 
σ_i = 2; ρ_g = 0.75; ρ_u = 0.75; σ_g = 1; σ_u = 1; σ_m = 0.1;

The equilibrium equations are:

$$x_t = \mathbb{E}_t x_{t+1} - \sigma(i_t-\mathbb{E}\pi_{t+1})+g_t$$
$$\pi_t = \kappa x_t+\beta \mathbb{E}_t \pi_{t+1}+u_t$$
$$i_t=\rho_i i_{t-1} + (1-\rho_i)(\phi_\pi \pi_t + \phi_x x_t) + \sigma_i \epsilon_{it}$$
$$g_t = \rho_g g_{t-1} + \sigma_g \epsilon_{gt}$$
$$u_t = \rho_u u_{t-1} + \sigma_u \epsilon_{ut}$$

Substituting expectations with outcomes plus errors, i.e. $x_{t+1}=\mathbb{E}_t \left[ x_{t+1} \right] + \eta_{t+1}$:

$$x_t = x_{t+1} - \eta_{xt+1} -\sigma(i_t -\pi_{t+1} +\eta_{\pi t+1})+g_t $$
$$\pi_t = \kappa x_t+\beta (\pi_{t+1} -\eta_{\pi t+1}) + u_t$$
$$i_t=\rho_i i_{t-1} + (1-\rho_i)(\phi_\pi \pi_t + \phi_x x_t) + \sigma_i \epsilon_{it}$$
$$g_t = \rho_g g_{t-1} + \sigma_g \epsilon_{gt}$$
$$u_t = \rho_u u_{t-1} + \sigma_u \epsilon_{ut}$$

The general form with expectational errors is:

$$\Gamma_0 \, y_t = \Gamma_1 \, y_{t-1} + c + \Psi z_t + \Pi \eta_t$$

I adopt the notation in which time arguments or subscripts
relate consistently to the information structure: variables dated t are always known at t.


Rearranging yields:

$$-x_{t} -\sigma \pi_{t} = -x_{t-1} -\sigma i_{t-1} +g_{t-1} -\eta_{xt} -\sigma \eta_{\pi t}$$
$$-\beta \pi_{t} = \kappa x_{t-1} -\pi_{t-1} + u_{t-1} -\beta \eta_{\pi t} $$
$$-(1-\rho_i) \phi_x x_t -(1-\rho_i) \phi_\pi \pi_t +i_t=\rho_i i_{t-1} +\sigma_i \epsilon_{it}$$
$$g_t = \rho_g g_{t-1} + \sigma_g \epsilon_{gt}$$
$$u_t = \rho_u u_{t-1} + \sigma_u \epsilon_{ut}$$

Define $y_t = \left[x_t \, \pi_t \, i_t \, g_t \, u_t\right]'$, $z_t = \left[\epsilon_{it} \, \epsilon_{gt} \, \epsilon_{ut} \right]'$ and $\eta_t = \left[\eta_{xt} \, \eta_{\pi t} \right]'$.

Now I am ready to write down the matrices:

In [2]:
Γ0 = [-1 -σ 0 0 0; 0 -β 0 0 0; -(1-ρ_i)ϕ_x -(1-ρ_i)ϕ_π 1 0 0; 0 0 0 1 0; 0 0 0 0 1]
Γ1 = [-1 0 -σ 1 0; κ -1 0 0 1; 0 0 ρ_i 0 0; 0 0 0 ρ_g 0; 0 0 0 0 ρ_u]
c = zeros(5)
Ψ = [0 0 0; 0 0 0; σ_i 0 0; 0 σ_g 0; 0 0 σ_u]
Π = [-1 -σ; 0 -β; 0 0; 0 0; 0 0];

## (b)

In [3]:
solution = gensys(Γ0, Γ1, c, Ψ, Π);
G1 = round(solution[1],12)

5×5 Array{Float64,2}:
  0.0  -0.0  -0.563271  1.18864   -1.09162 
 -0.0   0.0  -0.16626   0.591811   1.53767 
 -0.0   0.0   0.325679  0.518462   0.824592
  0.0   0.0   0.0       0.75      -0.0     
 -0.0   0.0  -0.0       0.0        0.75    

In [4]:
G2 = round(solution[3],12)

5×3 Array{Float64,2}:
 -2.25308  1.58486   -1.45549
 -0.66504  0.789081   2.05023
  1.30271  0.691283   1.09946
  0.0      1.0       -0.0    
 -0.0      0.0        1.0    

$$y_t = G_1 y_{t-1} + G_2 z_t$$

Observables:
$$Y_t = \left[x_t^{obs} \, \pi_t^{obs} \, i_t^{obs} \right]$$ with
$$Y_t = H(\theta)y_t + J(\theta)\nu_t$$ and:

In [5]:
H = [1 0 0 0 0; 0 1 0 0 0; 0 0 1 0 0]
J = eye(3) .* σ_m;

In [6]:
using Distributions
using Plots; gr()

N = 1000

y = zeros(5,N)
Y = zeros(3,N)

z = rand(MvNormal(zeros(3), eye(3)), N)
ν = rand(MvNormal(zeros(3), eye(3)), N)

for i in 2:size(y)[2]
    y[:,i] = G1*y[:,i-1] + G2*z[:,i]
    Y[:,i] = H*y[:,i] + J*ν[:,i]
end

In [7]:
plot(Y[1:3,:]', layout=3, title=["Output Gap" "Inflation" "Interest Rate"], legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 Output Gap 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,107.575 30.8694,118.454 31.1351,152.52 31.4008,140.222 31.6666,102.87 31.9323,97.1323 32.198,137.382 32.4637,137.432 32.7295,126.208 32.9952,135.268 
 33.2609,99.9584 33.5267,79.8566 33.7924,88.7372 34.0581,112.366 34.3238,104.945 34.5896,116.147 34.8553,138.178 35.121,143.671 35.3867,115.233 35.6525,114.21 
 35.9182,86.814 36.1839,68.6538 36.4496,78.4534 36.7154,124.501 36.9811,88.2344 37.2468,109.31 37.5125,94.37 37.7783,111.184 38.044,110.478 38.3097,119.965 
 38.5754,128.163 38.8412,117.846 39.1069,103.767 39.3726,101.215 39.6383,108.653 39.9041,91.2267 40.1698,139.536 40.4355,123.332 40.7012,125.242 40.967,149.374 
 41.2327,162.858 41.4984,157.084 41.7641,150.955 42.0299,132.028 42.2956,98.519 42.5613,101.674 42.827,95.0696 43.0928,117.279 43.3585,157.328 43.6242,139.856 
 43.8899,127.554 44.1557,154.875 44.4214,106.472 44.6871,76.3382 44.9528,71.9621 45.2186,110.74 45.4843,125.05 45.75,108.428 46.0157,108.735 46.2815,99.8018 
 46.5472,90.5231 46.8129,121.249 47.0786,138.125 47.3444,109.279 47.6101,119.701 47.8758,115.239 48.1415,91.5457 48.4073,110.404 48.673,106.058 48.9387,138.486 
 49.2044,117.589 49.4702,60.7993 49.7359,64.2567 50.0016,69.0405 50.2673,121.616 50.5331,154.867 50.7988,112.603 51.0645,94.0383 51.3302,112.484 51.596,114.205 
 51.8617,90.7369 52.1274,96.6812 52.3931,92.3023 52.6589,108.366 52.9246,124.14 53.1903,109.824 53.456,84.0585 53.7218,118.953 53.9875,74.0094 54.2532,64.0615 
 54.5189,89.3114 54.7847,78.0178 55.0504,127.188 55.3161,126.457 55.5818,119.073 55.8476,126.831 56.1133,146.193 56.379,124.744 56.6447,125.384 56.9105,127.233 
 57.1762,107.553 57.4419,93.9844 57.7076,82.3931 57.9734,97.0311 58.2391,104.844 58.5048,110.508 58.7705,83.4915 59.0363,125.412 59.302,131.994 59.5677,130.37 
 59.8334,123.941 60.0992,89.1151 60.3649,125.132 60.6306,129.879 60.8963,124.554 61.1621,115.475 61.4278,140.905 61.6935,157.626 61.9592,181.068 62.225,156.579 
 62.4907,172.285 62.7564,168.913 63.0221,133.382 63.2879,140.608 63.5536,104.385 63.8193,135.194 64.085,119.038 64.3508,105.326 64.6165,129.84 64.8822,113.154 
 65.1479,123.468 65.4137,108.785 65.6794,109.931 65.9451,107.188 66.2108,122.013 66.4766,101.912 66.7423,87.1185 67.008,80.5528 67.2737,79.4844 67.5395,41.2632 
 67.8052,84.0675 68.0709,108.63 68.3366,86.8123 68.6024,106.95 68.8681,86.1037 69.1338,66.8442 69.3995,69.9051 69.6653,73.3782 69.931,92.3601 70.1967,93.4341 
 70.4624,62.9914 70.7282,112.746 70.9939,83.685 71.2596,96.6892 71.5253,100.297 71.7911,125.952 72.0568,87.3685 72.3225,92.9627 72.5882,106.452 72.854,87.2534 
 73.1197,106.019 73.3854,83.289 73.6511,80.181 73.9169,84.364 74.1826,142.991 74.4483,127.273 74.714,97.5736 74.9798,97.0812 75.2455,79.5035 75.5112,89.7837 
 75.7769,94.044 76.0427,103.041 76.3084,112.077 76.5741,141.152 76.8398,127.572 77.1056,115.051 77.3713,125.258 77.637,105.872 77.9027,130.889 78.1685,124.5 
 78.4342,121.798 78.6999,107.753 78.9656,107.965 79.2314,123.192 79.4971,117.333 79.7628,92.2371 80.0285,64.9729 80.2943,47.5984 80.56,59.3197 80.8257,91.0729 
 81.0914,67.07 81.3572,96.6866 81.6229,108.509 81.8886,132.159 82.1543,90.3379 82.4201,112.46 82.6858,99.8694 82.9515,109.081 83.2172,108.294 83.483,99.6721 
 83.7487,95.1176 84.0144,125.753 84.2801,117.276 84.5459,116.691 84.8116,84.9281 85.0773,83.4827 85.343,91.7582 85.6088,101.443 85.8745,106.66 86.1402,94.6079 
 86.4059,58.3238 86.6717,46.5034 86.9374,65.8229 87.2031,83.5597 87.4688,97.4195 87.7346,110.268 88.0003,132.107 88.266,111.393 88.5317,107.369 88.7975,160.154 
 89.0632,146.432 89.3289,153.546 89.5946,119.622 89.8604,91.8551 90.1261,91.6579 90.3918,76.7396 90.6575,112.81 90.9233,174.416 91.189,133.197 91.4547,143.488 
 91.7204,150.105 91.

## (d)
The likelihood function is:
$$p(Y^T|\theta) = p(Y_0|\theta) \prod_{t=1}^T p(Y_t|Y^{t-1}, \theta)$$

To calculate conditional likelihoods I define:
$$y_{t|t-1} = G_1 y_{t-1|t-1}$$
$$P_{t|t-1} = G_1 P_{t-1|t-1} G_1' + G_2 G_2'$$
$$Y_{t|t-1} = H y_{t|t-1}$$
$$V_{t|t-1} = H P_{t|t-1} H' + J J'$$
$$p_t = p_{t-1} - \frac{1}{2}\left( \log |V_{t|t-1}| + (Y_t - Y_{t|t-1}) V^{-1}_{t|t-1} (Y_t - Y_{t|t-1}) \right)$$
$$y_{t|t} = y_{t|t-1} + P_{t|t-1} H' V^{-1}_{t|t-1}(Y_t - Y_{t|t-1})$$
$$P_{t|t} = P_{t|t-1} - P_{t|t-1} H' V^{-1}_{t|t-1} H P_{t|t-1}$$

In [8]:
y_forcast = zeros(5)
P_forcast = zeros(5, 5)
Y_forcast = zeros(3)
V_forcast = zeros(3, 3)

p = 0

y_nowcast = zeros(5)
P_nowcast = eye(5)

for i in 2:N-1
    y_forcast = G1 * y_nowcast
    P_forcast = G1 * P_nowcast * G1' + G2 * G2'
    Y_forcast = H * y_forcast
    V_forcast = H * P_forcast * H' + J * J'
    
    p += p - 1/2 * (log(det(V_forcast)) + (Y[:,i+1] - Y_forcast)' * inv(V_forcast) * (Y[:,i+1] - Y_forcast))[1]
    
    y_nowcast = y_forcast + P_forcast * H' * inv(V_forcast) * (Y[:,i+1] - Y_forcast)
    P_nowcast = P_forcast - P_forcast * H' * inv(V_forcast) * H * P_forcast
end

In [9]:
p

-1.3020323596898851e301

alternatively

In [14]:
y_nowcast = zeros(5)
P = eye(5)
a = zeros(3)
K = zeros(5, 3)

p = 0

0

In [15]:
for i in 2:N-1
    a = Y[:,i] - H * y_nowcast #innovation
    K = G1 * P * H' * inv(H*P*H' + J*J')
    
    p += p - 1/2 * (log(det(H*P*H' + J*J')) + a' * inv(H *P* H'+ J*J')  * a)[1]
    
    y_nowcast = G1 * y_nowcast + K * a
    P = G1*P*G1' * G2*G2' * G1*P*H'*inv(H*P*H'+J*J')*H*P*G1
    #(G1-K*H) * P * (G1-K*H)' + K * K' + G2 * G2'
end

In [16]:
p

NaN

## (e)

In [17]:
function eval_likelihood(;σ = 1, κ = 0.2, β = 0.99, ϕ_π = 1.25, ϕ_x = 0.25, ρ_i = 0.5, 
                        σ_i = 2, ρ_g = 0.75, ρ_u = 0.75, σ_g = 1, σ_u = 1, σ_m = 0.1)
    
    Γ0 = [-1 -σ 0 0 0; 0 -β 0 0 0; -(1-ρ_i)ϕ_x -(1-ρ_i)ϕ_π 1 0 0; 0 0 0 1 0; 0 0 0 0 1]
    Γ1 = [-1 0 -σ 1 0; κ -1 0 0 1; 0 0 ρ_i 0 0; 0 0 0 ρ_g 0; 0 0 0 0 ρ_u]
    c = zeros(5)
    Ψ = [0 0 0; 0 0 0; σ_i 0 0; 0 σ_g 0; 0 0 σ_u]
    Π = [-1 -σ; 0 -β; 0 0; 0 0; 0 0]

    solution = gensys(Γ0, Γ1, c, Ψ, Π);
    G1 = solution[1]
    G2 = solution[3]
    H = [1 0 0 0 0; 0 1 0 0 0; 0 0 1 0 0]
    J = σ_m
    
    y_forcast = zeros(5)
    P_forcast = zeros(5, 5)
    Y_forcast = zeros(3)
    V_forcast = zeros(5, 5)
    
    p = 0
    
    y_nowcast = zeros(5)
    P_nowcast = eye(5)

    for i in 2:N-1
        y_forcast = G1 * y_nowcast
        P_forcast = G1 * P_nowcast * G1' + G2 * G2'
        Y_forcast = H * y_forcast
        V_forcast = H * P_forcast * H' + J * J'
        
        p += p - 1/2 * (log(det(V_forcast)) + (Y[:,i+1] - Y_forcast)' * inv(V_forcast) * (Y[:,i+1] - Y_forcast))[1]
        
        y_nowcast = y_forcast + P_forcast * H' * inv(V_forcast) * (Y[:,i+1] - Y_forcast)
        P_nowcast = P_forcast - P_forcast * H' * inv(V_forcast) * H * P_forcast
    end
    p
end

eval_likelihood (generic function with 1 method)

In [18]:
plot(linspace(0.5,5,50), [eval_likelihood(σ=x)./1e300 for x in linspace(0.5,5,50)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -50 
 
 
 -40 
 
 
 -30 
 
 
 -20 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,15.3848 42.1437,12.3671 53.6836,9.8305 65.2236,7.7561 76.7636,6.13352 88.3036,4.95741 99.8436,4.22542 111.384,3.93701 122.924,4.09267 134.464,4.69347 
 146.004,5.74071 157.544,7.23573 169.084,9.17977 180.623,11.5739 192.163,14.4189 203.703,17.7154 215.243,21.4636 226.783,25.6634 238.323,30.3145 249.863,35.4162 
 261.403,40.9676 272.943,46.9674 284.483,53.414 296.023,60.3058 307.563,67.6405 319.103,75.416 330.643,83.6297 342.183,92.2787 353.723,101.36 365.263,110.871 
 376.803,120.807 388.343,131.166 399.883,141.944 411.423,153.136 422.963,164.739 434.503,176.748 446.043,189.159 457.583,201.969 469.123,215.171 480.663,228.762 
 492.203,242.737 503.743,257.09 515.283,271.818 526.823,286.914 538.363,302.375 549.903,318.194 561.443,334.366 572.983,350.887 584.523,367.751 596.063,384.952 
 
 "/>
 
 
 
 
 y1

In [19]:
plot(linspace(0.0,1,50), [eval_likelihood(κ=x)./1e300 for x in linspace(0.0,1,50)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -30 
 
 
 -20 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,9.18588 42.1437,6.91105 53.6836,5.33364 65.2236,4.36302 76.7636,3.93701 88.3036,4.01154 99.8436,4.55458 111.384,5.5423 122.924,6.95672 134.464,8.78403 
 146.004,11.0135 157.544,13.6369 169.084,16.6474 180.623,20.0399 192.163,23.8101 203.703,27.9546 215.243,32.4708 226.783,37.3564 238.323,42.6097 249.863,48.2292 
 261.403,54.2138 272.943,60.5626 284.483,67.2747 296.023,74.3496 307.563,81.7868 319.103,89.5859 330.643,97.7466 342.183,106.269 353.723,115.152 365.263,124.396 
 376.803,134.001 388.343,143.967 399.883,154.293 411.423,164.98 422.963,176.028 434.503,187.436 446.043,199.204 457.583,211.333 469.123,223.822 480.663,236.671 
 492.203,249.881 503.743,263.45 515.283,277.379 526.823,291.669 538.363,306.317 549.903,321.326 561.443,336.694 572.983,352.421 584.523,368.507 596.063,384.952 
 
 "/>
 
 
 
 
 y1

In [20]:
plot(linspace(0.0,1,50), [eval_likelihood(ϕ_x=x)./1e300 for x in linspace(0.0,1,50)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -18 
 
 
 -16 
 
 
 -14 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,16.3675 42.1437,11.6203 53.6836,8.11551 65.2236,5.73402 76.7636,4.37164 88.3036,3.93701 99.8436,4.34976 111.384,5.53898 122.924,7.44193 134.464,10.0029 
 146.004,13.1724 157.544,16.9063 169.084,21.165 180.623,25.9133 192.163,31.1193 203.703,36.7546 215.243,42.7935 226.783,49.2126 238.323,55.9911 249.863,63.11 
 261.403,70.5519 272.943,78.3014 284.483,86.344 296.023,94.667 307.563,103.258 319.103,112.108 330.643,121.205 342.183,130.54 353.723,140.107 365.263,149.896 
 376.803,159.902 388.343,170.117 399.883,180.536 411.423,191.154 422.963,201.965 434.503,212.965 446.043,224.15 457.583,235.515 469.123,247.058 480.663,258.776 
 492.203,270.664 503.743,282.72 515.283,294.942 526.823,307.327 538.363,319.873 549.903,332.578 561.443,345.44 572.983,358.458 584.523,371.629 596.063,384.952 
 
 "/>
 
 
 
 
 y1